In [1216]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np

In [1217]:
oraciones = pd.read_csv("data/oraciones.csv")
oraciones = oraciones[oraciones.personas.notnull()]
oraciones.head()

,noticia,medio,autor,titulo,fecha,lugar,oracion,sustantivos,adjetivos,verbos,personas,organizaciones,lugares,fechas,geopoliticos,eventos,polaridad,subjetividad
3,454.txt,News Online Today,NaN,LEADERSHIP CHANGE FOR PO,20090621,"ABILA, Krono","Following the unexpected death of Elian Karel,...","death,leadership,role",unexpected,"follow,take","Elian Karel,Silvia Marek",Kronos,NaN,NaN,NaN,NaN,0.050000,0.550000
4,454.txt,News Online Today,NaN,LEADERSHIP CHANGE FOR PO,20090621,"ABILA, Krono",Elian Karel died while in police custody on 19...,"police,custody",NaN,die,Elian Karel,NaN,NaN,19 June,NaN,NaN,0.000000,0.000000
10,544.txt,News Online Today,NaN,Activists Disrupt Traffic Near Tiskele Bend Fi...,20010831,"ABILA, Kronos","The protestors, members of the activist group ...","protestor,member,activist,group,sign,water,yea...",old,"hold,read,drill,poison,remember,become","Juliana,Juliana Vann","Protectors of Kronos,the Protectors of Kronos",NaN,"10 year old,1998",NaN,NaN,0.100000,0.200000
11,410.txt,News Online Today,NaN,LANDSLIDE VICTORY FOR PRESIDENT KAPELOU,20001002,"ABILA, Kronos",The people of Kronos have overwhelmingly elect...,people,new,elect,Dorel Kapelou II,NaN,NaN,NaN,Kronos,NaN,0.318182,0.727273
12,410.txt,News Online Today,NaN,LANDSLIDE VICTORY FOR PRESIDENT KAPELOU,20001002,"ABILA, Kronos",Former President Luis Araullo conceded defeat ...,"defeat,yesterday,return","former,early,strong","concede,come","Luis Araullo,Kapelou II",NaN,NaN,yesterday,NaN,NaN,0.177778,0.344444


In [1218]:
def long_df(columna_long, df): 
    df_long = pd.concat([df.loc[:, df.columns != columna_long],
                       df[columna_long].str.split(",", expand = True)], axis = 1)
    df_long=pd.melt(df_long, id_vars = [x for x in df.columns if x != columna_long],
                   value_name = columna_long)
    return df_long

In [1219]:
sustantivos_long = long_df("sustantivos",oraciones[["fecha","noticia", "sustantivos"]])
sustantivos_long=sustantivos_long.loc[:, sustantivos_long.columns != 'variable']
sustantivos_long=sustantivos_long[sustantivos_long["sustantivos"].notnull()]
sustantivos_long=sustantivos_long.drop_duplicates().sort_values("fecha").reset_index()
personas_long = long_df("personas",oraciones[["fecha","noticia", "personas"]])
personas_long=personas_long.loc[:, personas_long.columns != 'variable']
personas_long = personas_long[personas_long["personas"].notnull()]
personas_long["fl_fullname"] = personas_long.personas.str.contains(" ")
personas_long=personas_long[personas_long.fl_fullname==True]
personas_long=personas_long.drop(["fl_fullname"], axis = 1)
personas_long = personas_long.drop_duplicates().sort_values("fecha").reset_index()

In [ ]:
employees = pd.read_excel("data/EmployeeRecords.xlsx")
employees["full_name"] = employees["FirstName"] + " " + employees["LastName"]

In [ ]:
empleados_filtrados = pd.DataFrame()
for index, row in employees.iterrows():
    df_sub = oraciones[oraciones['personas'].str.contains(row['full_name'])]
    empleados_filtrados=empleados_filtrados.append(df_sub)

In [ ]:
entidades = pd.read_csv("data/entidades.csv", header=None)
entidades.columns = ["entidades"]

In [ ]:
oraciones_filtradas = pd.DataFrame()
for index, row in entidades.iterrows():
    df_sub = oraciones[oraciones['personas'].str.contains(row['entidades'])]
    oraciones_filtradas=oraciones_filtradas.append(df_sub)

In [ ]:
df = empleados_filtrados.append(oraciones_filtradas).drop_duplicates().sort_values('fecha')
df["cant_personas"] = df.personas.str.count(",") +1
df.to_csv("data/noticias_filtradas_entidades.csv", index = False)

#### Analisis de relaciones

In [ ]:
df_editado = df[df.cant_personas>1][["noticia","fecha","oracion", "sustantivos","personas", "organizaciones","cant_personas"]]
df_editado["personas"] = df_editado["personas"].str.replace("Elian Karel de Kronos,", "Elian Karel,")
df_editado["personas"] = df_editado["personas"].str.replace("ABILA", "Abila")
df_editado["personas"] = df_editado["personas"].str.replace(".", "")
df_editado["personas"] = df_editado["personas"].str.replace("'s", "")
df_editado["personas"] = df_editado["personas"].str.replace("'", "")
df_editado["personas"] = df_editado["personas"].str.replace("Ada Campo-Corren,", "Ada Campo-Corrente,")
df_editado["personas"] = df_editado["personas"].str.replace("Ada campo-Corrente,", "Ada Campo-Corrente,")
df_editado["personas"] = df_editado["personas"].str.replace("the Hank Fluss","Hank Fluss")
df_editado["personas"] = df_editado["personas"].str.replace("the Orhan Strum", "Orhan Strum")
df_editado["personas"] = df_editado["personas"].str.replace("Jeroen,","Jeroen Karel,")
df_editado["personas"] = df_editado["personas"].str.replace("Elian,","Elian Karel,")
df_editado["personas"] = df_editado["personas"].str.replace(",Elian",",Elian Karel")
df_editado["personas"] = df_editado["personas"].str.replace(",Elian Karel Karel",",Elian Karel")
df_editado["personas"] = df_editado["personas"].str.replace("Karel,Jeroen Karel", "Elian Karel,Jeroen Karel")
df_editado["personas"] = df_editado["personas"].str.replace("Juliana,", "Juliana Vann,")
df_editado["personas"] = df_editado["personas"].str.replace(",Fluss", ",Hank Fluss")
df_editado["personas"] = df_editado["personas"].str.replace("Sr-,", "Sr,")
df_editado["personas"] = df_editado["personas"].str.replace("Cato,","Rebecca Cato,")
df_editado["personas"] = df_editado["personas"].str.replace(",Sanjorge Jr,", ",Sanjorge Jr,")
df_editado["personas"] = df_editado["personas"].str.replace(",Sten Jr.,", ",Sanjorge Jr,")
df_editado["personas"] = df_editado["personas"].str.replace(",Sanjorge Jr.'s,", ",Sanjorge Jr,")
df_editado["personas"] = df_editado["personas"].str.replace(",Sten Sanjorge Jr.,", ",Sanjorge Jr,")
df_editado["personas"] = df_editado["personas"].str.replace(",Strum", ",Orhan Strum")
df_editado["personas"] = df_editado["personas"].str.replace("Marek,","Silvia Marek,") ### esta tiene que cambiarse
df_editado["personas"] = df_editado["personas"].str.replace("Silvia Silvia Marek,","Silvia Marek,") ### esta tiene que cambiarse
df_editado["personas"] = df_editado["personas"].str.replace("Reese", "Pondo Reese")
df_editado["personas"] = df_editado["personas"].str.replace("Kronosian", "")
df_editado["personas"] = df_editado["personas"].str.replace("Ingrid Barranca","Ingrid Barranco")
df_editado["personas"] = df_editado["personas"].str.replace("Ingrid Sanjorge","Ingrid Barranco")
df_editado["personas"] = df_editado["personas"].str.replace("Julian,", "Juliana Vann,")
df_editado["personas"] = df_editado["personas"].str.replace("Reynolds", "Joclyn Reynolds")
df_editado["personas"] = df_editado["personas"].str.replace("Lagos,", "Linda Lagos,")
df_editado["personas"] = df_editado["personas"].str.replace("Linda Linda Lagos,", "Linda Lagos,")
df_editado["organizaciones"] = ["POK" if "POK" in pers else pers for pers in df_editado["personas"]]
df_editado["personas"] = df_editado["personas"].str.replace("POK","")
df_editado["personas"] = df_editado["personas"].str.replace("Karel\x92s","")
df_editado["personas"] = df_editado["personas"].str.replace("Elodis","")
df_editado["personas"] = df_editado["personas"].str.replace(",Campo-Corrente", ",Ada Campo-Corrente")
df_editado["personas"] = df_editado["personas"].str.replace(",Cato", ",Rebecca Cato")
df_editado["personas"] = df_editado["personas"].str.replace("Osvaldo,", "Carmine Osvaldo,")
df_editado["personas"] = df_editado["personas"].str.replace("Nespola,","Cesare Nespola,")
df_editado["personas"] = df_editado["personas"].str.replace("Sten Sanjorge, Jr,","Sanjorge Jr,")
df_editado["personas"] = df_editado["personas"].str.replace("Sten Sanjorge Jr,","Sanjorge Jr,")
df_editado["personas"] = df_editado["personas"].str.replace("Sten Sanjorge Sr,","Sanjorge Sr,")
df_editado["personas"] = df_editado["personas"].str.replace(",Sten Sanjorge Jr",",Sanjorge Jr")
df_editado["personas"] = df_editado["personas"].str.replace("Sten Sanjorge, Sr,","Sanjorge Sr,")
df_editado["personas"] = df_editado["personas"].str.replace(",Sten Sanjorge, Sr",",Sanjorge Sr")
df_editado["personas"] = df_editado["personas"].str.replace("Sanjorge,Sten Jr,","Sanjorge Jr,")
df_editado["personas"] = df_editado["personas"].str.replace("Silvia Marek,Karel","Silvia Marek")
df_editado["personas"] = df_editado["personas"].str.replace("Elian Karel,Elian Karel,Juliana",
                                                            "Elian Karel, Juliana Vann")
df_editado["personas"] = df_editado["personas"].str.replace("Elian Karel,Juliana Vann,Silvia Marek,Juliana",
                                                            "Elian Karel,Juliana Vann,Silvia Marek")
df_editado["personas"] = df_editado["personas"].str.replace("Juliana Vann Vann", "Juliana Vann")
df_editado["personas"] = df_editado["personas"].str.replace(" Juliana Vann", "Juliana Vann")
df_editado["personas"] = ["Hank Fluss,Sanjorge Jr" if pers=="Sanjorge,Hank Fluss, Jr" else pers for pers in df_editado["personas"]]
df_editado["personas"] = ["Beatriz Fluss,Hank Fluss,Sanjorge Jr" if pers=="Jr,Sanjorge,Beatriz Fluss,Hank Fluss" else pers for pers in df_editado["personas"]]
df_editado["personas"] = ["Elian Karel,Sanjorge Jr" if pers=="Sanjorge Jr,Karel,Sanjorge" else pers for pers in df_editado["personas"]]
df_editado["personas"] = ["Hank Fluss,Sanjorge Sr" if pers=="Sanjorge,Hank Fluss" else pers for pers in df_editado["personas"]]
#df_editado["personas"] = ["Hank Fluss,Sanjorge Sr" if pers=="Sanjorge,Hank Fluss" else pers for pers in df_editado["personas"]]
#df_editado["personas"] = ["Hank Fluss,Sanjorge Sr" if pers=="Sanjorge,Hank Fluss" else pers for pers in df_editado["personas"]]
#df_editado["personas"] = ["Hank Fluss,Sanjorge Sr" if pers=="Sanjorge,Hank Fluss" else pers for pers in df_editado["personas"]]



In [ ]:
df_editado.head()

In [ ]:
personas = df_editado["personas"].str.split(",", expand=True)
df_editado = pd.merge(df_editado, personas, left_index=True, right_index=True)

In [ ]:
personas_unicas = []
for index, row in personas.iterrows():
    [personas_unicas.append(r) for r in row if r is not None]

In [ ]:
personas_unicas.sort()
set(personas_unicas)

In [ ]:
df_editado[df_editado[["noticia", 0,1,2,3,4,5]].isin(["Sanjorge"]).any(axis = 1)]